In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
gtzan = '/content/drive/MyDrive/MiniProject/GTZANUpdated'
genre_folders=os.listdir(gtzan)
print("Genres to classify are:", *genre_folders, sep='\n')

Genres to classify are:
blues
classical
pop
disco
metal
hiphop
reggae
jazz
rock
country


In [60]:
model_to_load = '/content/drive/MyDrive/MiniProject/Models/'
print("The trained model files available are:")
print(*[i for i in os.listdir(model_to_load) if i.endswith(".keras") or i.endswith(".h5")], sep = '\n')

The trained model files available are:
cnnmodel.h5
cnnmodel2.h5
cnn-stft-15-updated.h5
cnn-mpcc-15-updated.h5
cnn-mpcc-20-updated.h5
cnn-stft-20-updated.h5
cnn-mpcc-15-old.h5
cnn-mpcc-20-old.h5
cnn-stft-15-old.h5
cnn-stft-100-15-old.h5
cnn-stft-100-20-old.h5
cnn-mpcc-100-15-updated.keras
cnn-mpcc-100-15-updated.h5
cnn-mpcc-100-20-updated.keras
cnn-mpcc-100-20-updated.h5


In [61]:
mfcc_modelname = "cnn-mpcc-100-20-updated.keras"
stft_modelname = "cnn-stft-100-20-old.h5"
mfcc_model = load_model(os.path.join(model_to_load, mfcc_modelname))
stft_model = load_model(os.path.join(model_to_load, stft_modelname))

In [62]:
import librosa

def extract_mfcc_single(file_path, max_len=200):
    y, sr = librosa.load(file_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    if mfcc.shape[1] < max_len:
        mfcc = np.pad(mfcc, ((0, 0), (0, max_len - mfcc.shape[1])), mode='constant')
    else:
        mfcc = mfcc[:, :max_len]
    return mfcc


In [63]:
filename = "Copy of classical.00001.wav"
''' PUT OWN FILE NAME HERE ^^^^^^^^^ '''
filepath = os.path.join('/content/drive/MyDrive/MiniProject/Testfiles/', filename)
test_features = extract_mfcc_single(filepath)

In [64]:
def predict_genre_mfcc(file_path, model, genre_folders, max_len=200):
    mfcc = extract_mfcc_single(file_path, max_len)
    mfcc = mfcc[np.newaxis, ..., np.newaxis]
    prediction = model.predict(mfcc)
    predicted_index = np.argmax(prediction, axis=1)[0]
    predicted_genre = genre_folders[predicted_index]
    return predicted_genre

In [69]:
predicted_genre = predict_genre_mfcc(filepath, mfcc_model, genre_folders)


1/1 [==============================] - 0s 39ms/step


In [70]:

def extract_and_pad_stft_features(file_path, max_len=200):
    y, sr = librosa.load(file_path, sr=None)
    stft = np.abs(librosa.stft(y))
    if stft.shape[1] < max_len:
        stft = np.pad(stft, ((0, 0), (0, max_len - stft.shape[1])), mode='constant')
    else:
        stft = stft[:, :max_len]
    return stft

def predict_genre_stft(file_path, model, genre_folders, max_len=200):
    stft = extract_and_pad_stft_features(file_path, max_len)
    stft = stft[np.newaxis, ..., np.newaxis]
    prediction = model.predict(stft)
    predicted_index = np.argmax(prediction, axis=1)[0]
    predicted_genre = genre_folders[predicted_index]
    return predicted_genre

predicted_genre_stft = predict_genre_stft(filepath, stft_model, genre_folders)


1/1 [==============================] - 0s 415ms/step


In [71]:
print("MFCC Prediction:", predicted_genre)
print("STFT Prediction:", predicted_genre_stft)

MFCC Prediction: classical
STFT Prediction: classical


In [ ]:
%%script false --no-raise-error
with open('/content/drive/MyDrive/MiniProject/Models/mfcc-featuresX.pkl', 'rb') as file:
    mfcc_X = pickle.load(file)
with open('/content/drive/MyDrive/MiniProject/Models/mfcc-featuresY.pkl', 'rb') as file:
    mfcc_y = pickle.load(file)